In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import gc
from gensim.models import Word2Vec
from collections import Counter

In [2]:
train = pd.read_csv('sortedtrain.csv')

In [ ]:
lst = Counter(train['app'])

In [8]:
len(lst)

706

In [9]:
rare = { k:-1 for k, v in lst.items() if v <= 5}

In [10]:
len(rare)

248

In [11]:
train['app'] = train['app'].replace(rare)

In [3]:
train['app'] = train['app'].astype(str)

In [12]:
lst = train.groupby('ip').app.apply(pd.Series.tolist).tolist()

In [13]:
model_app = Word2Vec(lst, size = 150, window = 3, min_count=1, workers = 4)

In [16]:
#model_app.save('vec_app.txt')

In [12]:
lst = Counter(train['channel'])

In [13]:
len(lst)

202

In [14]:
rare = { k:-1 for k, v in lst.items() if v <= 5}

In [15]:
len(rare)

17

In [16]:
train['channel'] = train['channel'].replace(rare)

In [8]:
train['channel'] = train['channel'].astype(str)

In [26]:
lst = train.groupby('ip').channel.apply(pd.Series.tolist).tolist()

In [27]:
model_channel = Word2Vec(lst, size = 50, window = 3, min_count=1, workers = 4)

In [28]:
#model_channel.save('vec_channel.txt')

In [3]:
lst = Counter(train['os'])

In [4]:
len(lst)

800

In [5]:
rare = { k:-1 for k, v in lst.items() if v <= 5}

In [6]:
len(rare)

460

In [11]:
a = list(rare.keys())[400:]
a = dict(zip(a,[-1]*60))

In [12]:
train['os'] = train['os'].replace(a)

In [13]:
train['os'] = train['os'].astype(str)

In [14]:
lst = train.groupby('ip').os.apply(pd.Series.tolist).tolist()

In [15]:
model_os = Word2Vec(lst, size = 50, window = 3, min_count=1, workers = 4)

In [17]:
#model_os.save('vec_os.txt')

In [ ]:
train

In [26]:
lst = Counter(train['device'])

In [27]:
len(lst)

3475

In [28]:
rare = { k:-1 for k, v in lst.items() if v <= 5}

In [29]:
len(rare)

1927

In [48]:
a = list(rare.keys())[1800:]
a = dict(zip(a,[-1]*127))

In [49]:
train['device'] = train['device'].replace(a)

In [37]:
train['device'] = train['device'].astype(str)

In [38]:
lst = train.groupby('ip').device.apply(pd.Series.tolist).tolist()

In [39]:
model_device = Word2Vec(lst, size = 50, window = 3, min_count=1, workers = 4)

In [40]:
#model_device.save('vec_device.txt')

In [3]:
train['day'] = pd.to_datetime(train.click_time).dt.day.astype('float32')
train['hour'] = pd.to_datetime(train.click_time).dt.hour.astype('float32')

In [4]:
train = train.drop(columns = ['click_time'])

In [5]:
train['count'] = train.groupby(['ip','day','hour'])['ip'].transform('count')

In [6]:
train = train.drop(columns = ['day','hour','ip'])

In [51]:
#train.to_csv('ready.csv', index = False)

In [2]:
test = pd.read_csv('test.csv')

In [3]:
test['os'] = test['os'].astype(str)
test['app'] = test['app'].astype(str)
test['device'] = test['device'].astype(str)
test['channel'] = test['channel'].astype(str)

In [4]:
test['day'] = pd.to_datetime(test.click_time).dt.day.astype('float32')
test['hour'] = pd.to_datetime(test.click_time).dt.hour.astype('float32')

In [5]:
test['count'] = test.groupby(['day','hour','ip'])['ip'].transform('count')

In [6]:
test = test.drop(columns = ['click_time','day','hour','ip'])

In [56]:
lst_app = list(model_app.wv.vocab)
lst_os = list(model_os.wv.vocab)
lst_channel = list(model_channel.wv.vocab)
lst_device = list(model_device.wv.vocab)

In [9]:
lst_app1 = list(set(list(test['app'])))
lst_device1 = list(set(list(test['device'])))
lst_os1 = list(set(list(test['os'])))
lst_channel1 = list(set(list(test['channel'])))

In [ ]:
l_app = {}
l_device = {}
l_os = {}
l_channel = {}
for i in lst_app1:
    if i not in lst_app:
        l_app[i] = '-1'
        
for i in lst_device1:
    if i not in lst_device:
        l_device[i] = '-1'

for i in lst_os1:
    if i not in lst_os:
        l_os[i] = '-1'

for i in lst_channel1:
    if i not in lst_channel:
        l_channel[i] = '-1'

In [11]:
print(len(l_app))
print(len(l_device))
print(len(l_os))
print(len(l_channel))

58
747
121
2


In [12]:
test['os'] = test['os'].replace(l_os)
test['channel'] = test['channel'].replace(l_channel)
test['app'] = test['app'].replace(l_app)

In [18]:
a = list(l_device.keys())[600:747]
a = dict(zip(a,[-1]*147))

In [62]:
test['device'] = test['device'].replace(l_device)

In [63]:
#test.to_csv('testready.csv', index = False)

# Start here!

In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
import gc
from gensim.models import Word2Vec
from collections import Counter
from sklearn import metrics

In [2]:
train = pd.read_csv('ready.csv')

In [3]:
test = pd.read_csv('testready.csv')

In [4]:
model_app = Word2Vec.load('vec_app.txt')
model_os = Word2Vec.load('vec_os.txt')
model_device = Word2Vec.load('vec_device.txt')
model_channel = Word2Vec.load('vec_channel.txt')

In [5]:
train['app'] = train['app'].astype(str)
train['device'] = train['device'].astype(str)
train['os'] = train['os'].astype(str)
train['channel'] = train['channel'].astype(str)

In [6]:
test['app'] = test['app'].astype(str)
test['device'] = test['device'].astype(str)
test['os'] = test['os'].astype(str)
test['channel'] = test['channel'].astype(str)

In [7]:
def generate_batch(data, batch_size, num_epochs, shuffle=True):
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    l = 0
    for epoch in range(num_epochs):
        l += 1
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [8]:
train1 = train.iloc[0:3698078,:]
train2 = train.iloc[3698078:3698078*2,:]
train3 = train.iloc[3698078*2:3698078*3,:]
train4 = train.iloc[3698078*3:3698078*4,:]
train5 = train.iloc[3698078*4:3698078*5,:]

In [9]:
print(train1['is_attributed'].mean())
print(train2['is_attributed'].mean())
print(train3['is_attributed'].mean())
print(train4['is_attributed'].mean())
print(train5['is_attributed'].mean())

0.0012249606417171297
0.0014837437176825367
0.001479687556617248
0.0014642741445691518
0.0012468639114696877


In [10]:
del train 
gc.collect()

313

In [11]:
batches1 = generate_batch(train1, 10000, 1)
batches2 = generate_batch(train2, 10000, 1)
batches3 = generate_batch(train3, 10000, 1)
batches4 = generate_batch(train4, 10000, 1)
batches5 = generate_batch(train5, 10000, 1)
batches6 = generate_batch(train1, 10000, 1)
batches7 = generate_batch(train2, 10000, 1)
batches8 = generate_batch(train3, 10000, 1)
batches9 = generate_batch(train4, 10000, 1)
batches10 = generate_batch(train5, 10000, 1)
batches11 = generate_batch(train1, 10000, 1)
batches12 = generate_batch(train2, 10000, 1)
batches13 = generate_batch(train3, 10000, 1)
batches14 = generate_batch(train4, 10000, 1)
batches15 = generate_batch(train5, 10000, 1)
batches16 = generate_batch(train1, 10000, 1)
batches17 = generate_batch(train2, 10000, 1)
batches18 = generate_batch(train3, 10000, 1)
batches19 = generate_batch(train4, 10000, 1)
batches20 = generate_batch(train5, 10000, 1)
batches21 = generate_batch(train1, 10000, 1)
batches22 = generate_batch(train2, 10000, 1)
batches23 = generate_batch(train3, 10000, 1)
batches24 = generate_batch(train4, 10000, 1)
batches25 = generate_batch(train5, 10000, 1)

In [12]:
batch_bag1 = [batches1,batches2,batches3,batches4,batches5]
batch_bag2 = [batches6,batches7,batches8,batches9,batches10]
batch_bag3 = [batches11,batches12,batches13,batches14,batches15]
batch_bag4 = [batches16,batches17,batches18,batches19,batches20]
batch_bag5 = [batches21,batches22,batches23,batches24,batches25]

In [13]:
batch_bags = [batch_bag1,batch_bag2,batch_bag3,batch_bag4,batch_bag5]

In [14]:
input_x = tf.placeholder(tf.float32, [None, 301])
input_y = tf.placeholder(tf.float32, [None, 2])

In [15]:
w1 = tf.Variable(tf.random_normal([301, 301], stddev = 0.05), name = 'w1')
b1 = tf.Variable(tf.random_normal([301], stddev = 0.05), name = 'b1')
w2 = tf.Variable(tf.random_normal([301, 150], stddev = 0.05), name = 'w2')
b2 = tf.Variable(tf.random_normal([150], stddev = 0.05), name = 'b2')
w3 = tf.Variable(tf.random_normal([150, 150], stddev = 0.05), name = 'w3')
b3 = tf.Variable(tf.random_normal([150], stddev = 0.05), name = 'b3')
w4 = tf.Variable(tf.random_normal([150, 150], stddev = 0.05), name = 'w4')
b4 = tf.Variable(tf.random_normal([150], stddev = 0.05), name = 'b4')
w5 = tf.Variable(tf.random_normal([150, 2], stddev = 0.05), name = 'w5')
b5 = tf.Variable(tf.random_normal([2], stddev = 0.05), name = 'b5')

In [16]:
layer1 = tf.nn.xw_plus_b(input_x, w1, b1, name = 'layer1')
layer1 = tf.nn.relu(layer1)
layer1 = tf.nn.dropout(layer1,0.8)
layer2 = tf.nn.xw_plus_b(layer1, w2, b2, name = 'layer2')
layer2 = tf.nn.relu(layer2)
layer3 = tf.nn.xw_plus_b(layer2, w3, b3, name = 'layer3')
layer3 = tf.nn.relu(layer3)
layer4 = tf.nn.xw_plus_b(layer3, w4, b4, name = 'layer4')
layer4 = tf.nn.relu(layer4)
layer5 = tf.nn.xw_plus_b(layer4, w5, b5, name = 'layer5')
prediction = tf.nn.softmax(layer5)

In [17]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = layer5, labels = input_y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.0016).minimize(loss)
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(input_y, 1), tf.argmax(prediction, 1)), tf.float32))

In [18]:
test_blocks = generate_batch(test, 20000, 1, shuffle = False)

In [19]:
print(test.columns)
print(train1.columns)

Index(['app', 'device', 'os', 'channel', 'count'], dtype='object')
Index(['app', 'device', 'os', 'channel', 'is_attributed', 'count'], dtype='object')


In [20]:
(int((len(train1)-1)/10000) + 1) * 5

1850

In [21]:
submission = pd.read_csv("sample_submission.csv")

In [22]:
init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    
    sess.run(init_op)
    
    print('Start!')
    i = 0
    for batch_bag in batch_bags:
        i += 1
        print('Epoch ' + str(i) + ' Start!')
        avg_loss = 0
        avg_acc = 0
        avg_auc = 0
        for batches in batch_bag:
            for batch in batches:
                batch = pd.DataFrame(batch, columns = ['app','device','os','channel','is_attributed','count'])
                x_batch = batch.loc[:, batch.columns != 'is_attributed']
                y_batch = batch.loc[:, batch.columns == 'is_attributed']
                x_batch = pd.concat([x_batch, pd.DataFrame(model_app.wv[x_batch['app']]), pd.DataFrame(model_channel.wv[x_batch['channel']]), pd.DataFrame(model_os.wv[x_batch['os']]), pd.DataFrame(model_device.wv[x_batch['device']])],axis = 1)
                x_batch = x_batch.drop(columns = ['app','os','device','channel'])
                y_batch['is_not_attributed'] = 1 - y_batch['is_attributed']
                _,c, acc, pred = sess.run([optimizer, loss, accuracy, prediction],feed_dict = {input_x: x_batch, input_y: y_batch})
                avg_loss += c
                avg_acc += acc
                avg_auc += metrics.roc_auc_score(y_batch['is_attributed'].astype(int), pred[:,0])
        print('Average loss is: ' + str(avg_loss/1850) + ', Average accuracy is: ' + str(avg_acc/1850) + ', Average AUC is: ' + str(avg_auc/1850))
    
    print('Prediction Start!')
    
    df = pd.DataFrame()
    for block in test_blocks:
        block = pd.DataFrame(block, columns = ['app', 'device', 'os', 'channel', 'count'])
        block = pd.concat([block, pd.DataFrame(model_app.wv[block['app']]), pd.DataFrame(model_channel.wv[block['channel']]), pd.DataFrame(model_os.wv[block['os']]), pd.DataFrame(model_device.wv[block['device']])],axis = 1)
        block = block.drop(columns = ['app','device','os','channel'])
        pred = sess.run(prediction, feed_dict = {input_x: block})
        df = df.append(pd.DataFrame(pred))
    
    print('Finish!')

Start!
Epoch 1 Start!


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Average loss is: 0.10638421100268543, Average accuracy is: 0.9972904701812847, Average AUC is: 0.8328819993442493
Epoch 2 Start!
Average loss is: 0.016555580908206066, Average accuracy is: 0.9985921183147946, Average AUC is: 0.8861875331935268
Epoch 3 Start!
Average loss is: 0.006417900356401161, Average accuracy is: 0.9988564408469844, Average AUC is: 0.9198317908164891
Epoch 4 Start!
Average loss is: 0.006119146412490188, Average accuracy is: 0.9988606894982828, Average AUC is: 0.9318202171628155
Epoch 5 Start!
Average loss is: 0.005927646759570249, Average accuracy is: 0.9988624161320764, Average AUC is: 0.9378347502072758
Prediction Start!
Finish!


In [81]:
df.round().mean()

click_id         9996.844930
is_attributed       0.000584
dtype: float64

In [121]:
df.round().mean()

0    0.000499
1    0.892858
dtype: float32

In [142]:
df.round().mean()

0    0.000527
1    0.892858
dtype: float32

In [23]:
df.round().mean()

0    0.000537
1    0.892858
dtype: float32

In [24]:
df.columns = ['is_attributed','b']

In [25]:
submission['is_attributed'] = np.array(df['is_attributed'])
submission.to_csv("submission.csv", index=False)